In [1]:
import tensorflow_datasets as tfds
from tensorflow.keras.layers import *
from tensorflow.keras import Model
import tensorflow as tf

from sklearn.model_selection import train_test_split
import numpy as np

## Load data

In [2]:
dataset, info = tfds.load(
    "uc_merced",
    split=['train'],
    with_info=True,
    batch_size=-1
)

In [3]:
info

tfds.core.DatasetInfo(
    name='uc_merced',
    full_name='uc_merced/2.0.0',
    description="""
    UC Merced is a 21 class land use remote sensing image dataset, with 100 images
    per class. The images were manually extracted from large images from the USGS
    National Map Urban Area Imagery collection for various urban areas around the
    country. The pixel resolution of this public domain imagery is 0.3 m.
    
    While most images are 256x256 pixels, there are 44 images with different shape.
    """,
    homepage='http://weegee.vision.ucmerced.edu/datasets/landuse.html',
    data_path='dataset/uc_merced/2.0.0',
    file_format=tfrecord,
    download_size=317.07 MiB,
    dataset_size=238.63 MiB,
    features=FeaturesDict({
        'filename': Text(shape=(), dtype=string),
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=21),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
  

## Data processing

In [4]:
def preprocess(image):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, (224, 224))
    return image

In [5]:
images = []
for i in tfds.as_numpy(dataset[0])[0]:
    images.append(preprocess(i))

images = np.array(images)
labels = tfds.as_numpy(dataset[0])[1].astype('uint32')

In [6]:
images.shape, labels.shape

((2100, 224, 224, 3), (2100,))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1890, 224, 224, 3), (210, 224, 224, 3), (1890,), (210,))

## Build model

In [8]:
def dense_block(x, iteration):
    for _ in range(iteration):
        y = x
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2D(128, (1,1), strides=(1,1), padding="same", use_bias=False)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2D(32, (3,3), strides=(1,1), padding="same", use_bias=False)(x)
        x = concatenate([x, y])
    return x

In [9]:
def transition_layer(x):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(x.shape[-1]// 2, (1,1), strides=(1,1), padding="same", use_bias=False)(x)
    x = AveragePooling2D((2,2), strides=(2,2), padding="same")(x)
    return x

In [10]:
def DenseNet121(input_shape=(224,224,3), num_classes=1000):
    x_input = Input(input_shape)
    x = Conv2D(64, (7,7), strides=(2,2), padding="same", use_bias=False)(x_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool2D((3,3), strides=(2,2), padding="same")(x)

    for iteration in [6, 12, 24, 16]:
        y = dense_block(x, iteration)
        x = transition_layer(y)

    x = GlobalAveragePooling2D()(y)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x_output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=x_input, outputs=x_output, name='DenseNet121')
    return model

In [11]:
dense_model = DenseNet121(num_classes=21)
dense_model.summary()

Model: "DenseNet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9408        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                       

In [12]:
dense_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
dense_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50
)

Epoch 1/50
60/60 [==============================] - 108s 448ms/step - loss: 2.2806 - accuracy: 0.2984 - val_loss: 3.8438 - val_accuracy: 0.1048
Epoch 2/50
60/60 [==============================] - 21s 348ms/step - loss: 1.7331 - accuracy: 0.4524 - val_loss: 11.0719 - val_accuracy: 0.0905
Epoch 3/50
60/60 [==============================] - 21s 344ms/step - loss: 1.6528 - accuracy: 0.4571 - val_loss: 7.2249 - val_accuracy: 0.0190
Epoch 4/50
60/60 [==============================] - 21s 344ms/step - loss: 1.5445 - accuracy: 0.4915 - val_loss: 6.6048 - val_accuracy: 0.0857
Epoch 5/50
60/60 [==============================] - 21s 344ms/step - loss: 1.2941 - accuracy: 0.5667 - val_loss: 3.6615 - val_accuracy: 0.2238
Epoch 6/50
60/60 [==============================] - 21s 346ms/step - loss: 1.2222 - accuracy: 0.5921 - val_loss: 9.1132 - val_accuracy: 0.2429
Epoch 7/50
60/60 [==============================] - 21s 346ms/step - loss: 1.1195 - accuracy: 0.6206 - val_loss: 4.7636 - val_accuracy: 0.23